# Reduce overfitting
* More data
* 限制模型的复杂性（更浅层的网络或者正则化）
* dropout
* 数据增强
* early stopping

# 1.训练集&验证集&测试集

In [ ]:
train_db ,val_db = torch.utils.data.random_split(train_db,[50000,10000])

# 2.正则化

* 在定义优化器的时候设定weigth_decay，即L2范数前面的参数<br/>
optimizer = torch.optim.SGD(net.parameters(),lr = learning_rate,weight_decay = 0.01)

In [1]:
# L1-regualarization
regularization_loss = 0
for param in model.parameters():
    regularization_loss+=torch.sum(torch.abs(param))
classify_loss = criteon(logits,target)
loss=classify_loss+0.01*regularization_loss
optimizer.zero_grad()
loss.backward()
optimizer.step()

NameError: name 'model' is not defined

# 3.动量（惯性的意思）
args.momentum

In [2]:
optimizer = torch.optim.SGD(mdoel.parameters(),args.lr,momentum = args.momentum,weight_decay = args.weight_decay)

NameError: name 'torch' is not defined

# 4.学习率衰减
* torch.optim.lr_scheduler中提供了基于多种epoch数目，调整学习率的方法。

## 4.1ReduceLROnPlateau
* torch.optim.lr_scheduler.ReduceLROnPlateau：基于测量指标对学习率进行动态的下降

In [6]:
import torch
torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer,mode = 'min',factor = 0.1,patience = 10,verbose = False,threshold = 0.0001,threshold_model = 'rel',cooldown = 0,min_lr = 0,eps = 1e-8)

NameError: name 'optimizer' is not defined

## 训练过程中，optimizer会把learning_rate交给scheduler管理
* 当指标（比如loss）连续patience次还没有改进的时候，需要降低学习率，factor为每次下降的比例。
* scheduler.step(loss_val)每调用一次就会监听一次loss_val

In [ ]:
optimizer = torch.optim.SGD(model.parameters(),args.lr,momentum = args.momentum,weight_decay = args.weight_decay)
scheduler = ReduceLROnPlateau(optimizer,mode='min',factor = 0.1)


for epoch in xrange(args.start_epoch,args.epochs):
    train(train_loader,model,criterion,optimizer,epoch)
    result_avg,loss_val = validate(val_loader,model,criterion,epoch)
    scheduler.step(loss_val)

## 4.2StepLR
* torch.optim.lr_scheduler.StepLR:基于epoch

* torch.optim.lr_scheduler.StepLR(optimizer,step_size,gamma = 0.1,last_epoch = -1)

In [ ]:
# 当epoch每经过step_size后，学习率都变为初始学习率的gamma倍
# lr = 0.05  if epoch <30
# lr = 0.005 if 30<=epoch <=60
# lf = 0.005 if 60<=epoch <=90
scheduler = StepLR(optimizer,step_size = 30,gamma = 0.1)
for epoch in range(1000):
    scheduler.step()
    train()
    validate()

# 5.EarlyStopping

# 6.Dropout随机失活
* 遍历每一层，设置消除神经网络中的节点概率，得到精简后的一个样本
* torch.nn.Dropout(p = dropout_prob)
* p表示的是删除节点的比例（tip:tensorflow中keep_prob表示保留节点数的比例）

In [7]:
net_dropped = torch.nn.Sequential(
    torch.nn.Linear(784,200),
    torch.nn.Dropout(0.8),
    torch.nn.ReLU(),
    torch.nn.Linear(200,200),
    torch.nn.Dropout(0.5),
    torch.nn.ReLU(),
    torch.nn.Linear(200,10)
)

# 测试阶段无需使用dropOut
* 在train之前执行net_dropped.train()相当于启动dropout
* 测试之前执行net_dropped.eval()相当于不启动dropout

In [ ]:
for epoch in range(epochs):
#     train
    net_dropped.train()
    for batch_idx,(data,target) in enumerater(train_loader):
        ...
    net_dropped.eval()
    test_loss = 0
    correct = 0
    for data,target in test_loader:
        ...